Notebook for Pennylane stuff and looking at single events

In [ ]:
from datetime import datetime as dt

import matplotlib.pyplot as plt
import numpy
import pennylane as qml
from IPython.display import HTML
from matplotlib import rcParams
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MultipleLocator, StrMethodFormatter
from my_favorite_things import bar_count
from pennylane import numpy as np

from scripts.constants import SYM_TRUE_BS_DICT, EVT_DIR
from scripts.qc_utilities import format_p4s, get_Jijs_Pijs, swap
from scripts.pennylane_algs import QAOA, MAQAOA, XQAOA, FALQON

# Effects of Finite Shots on a Simple, Random Circuit

In [2]:
# build circuit
def qc(thetas):
    qml.Hadamard(0)
    qml.Hadamard(1)
    qml.Hadamard(2)
    qml.CNOT([0, 1])
    qml.CNOT([1, 2])
    qml.RY(thetas[0], 0)
    qml.RX(3.2 * thetas[1], 2)
    qml.RZ(thetas[2] ** 2, 0)
    qml.CRX(thetas[3], [1, 2])
    qml.CRZ(thetas[4], [2, 0])
    qml.CNOT([0, 1])
    qml.Hadamard(0)
    qml.Hadamard(1)
    qml.Hadamard(2)

    return qml.expval(qml.PauliX(0) @ qml.PauliZ(1) @ qml.PauliY(2))


# Create circuit qnodes for different amount of shots, None == infinite
shot_vals = [50, 100, 1000, 5000, 10000, 50_000, 100_000, 1_000_000, None]
devs = [qml.device("default.qubit", wires=3, shots=shots) for shots in shot_vals]
qcs = [qml.QNode(qc, dev) for dev in devs]

In [ ]:
qml.draw_mpl(qc)(range(5));

In [ ]:
# Run circuits for 250 steps each
steps = 250

costss = []
for shot_val, qc in zip(shot_vals, qcs):
    t0 = dt.now()
    opt = qml.AdamOptimizer()
    theta = np.array([0.5] * 5, requires_grad=True)
    costs = []
    for x in range(steps):
        theta, cost = opt.step_and_cost(qc, theta)
        costs.append(cost)

    t1 = dt.now()
    if shot_val is None:
        print(f"{'None':>7} -- {(t1 - t0).total_seconds():.3f}s")
    else:
        print(f"{shot_val:>7} -- {(t1 - t0).total_seconds():.3f}s")

    costss.append(costs)

In [ ]:
# Plot expectation value vs step
fig, ax = plt.subplots(figsize=(16, 10))

for costs, shot_val in zip(costss, shot_vals):
    if shot_val is None:
        ax.plot(costs, label=f"{'infinite':>15}", c="k")
    else:
        ax.plot(costs, label=f"{shot_val:>11,}")

ax.set_xlim(0, len(costs))
ax.set_xlabel("Shots")
ax.set_ylabel(r"Cost $\langle XZY\rangle$")
ax.legend(title="Shots", title_fontsize=18, fontsize=15)

---

# Comparing Single Events

## Import data

In [3]:
event = "ttbar"
correct_bs = SYM_TRUE_BS_DICT[event]

# Get all data for all events
p4s = numpy.load(EVT_DIR / f"{event}_parton.npy")
num_fsp, p4s, invms = format_p4s(p4s, return_extra=True)
Jijs, Pijs = get_Jijs_Pijs(p4s)

# Choose a highly boosted event
high_invm_inds = numpy.where(invms > 2.5)[0]
invm_ind = 12
ind = high_invm_inds[invm_ind]

# Get event-specific data
p4 = p4s[ind]
Jij, Pij = Jijs[ind], Pijs[ind]
norm_Jij = Jij / Jij.max()
norm_lmbda = Jij + np.min(Jij) / (2 * np.max(Pij)) * Pij


## Run For different algorithms

In [ ]:
# QAOA
qaoa = QAOA(norm_Jij, depth=5, steps=1000, optimizer="adam", prec=1e-6)
qaoa.optimize(print_it=True)

In [ ]:
# ma-QAOA
maqaoa = MAQAOA(norm_Jij, depth=5, steps=1000, optimizer="adam", prec=1e-6)
maqaoa.optimize(print_it=True)

In [ ]:
# XQAOA
xqaoa = XQAOA(norm_Jij, depth=5, steps=1000, optimizer="adam", prec=1e-6)
xqaoa.optimize(print_it=True)


In [ ]:
# FALQON
falqon = FALQON(Jij=norm_Jij, depth=500)
falqon.run(print_it=True)


In [ ]:
algs = [qaoa, maqaoa, xqaoa, falqon]
syms = [True, True, True, True]
rows = len(algs)

fig = plt.figure(figsize=(20, len(algs) * 6))
subfigs = fig.subfigures(rows, 1)


for row in range(rows):
    alg = algs[row]
    sym = syms[row]
    fig = subfigs[row]
    axes = fig.subplots(1, 2, gridspec_kw={"width_ratios": [25, 11]})

    try:
        probs = alg.get_probs(as_dict=True)
    except AttributeError:
        tmp = qml.QNode(alg._probs_circuit, alg.device)
        probs = tmp(*alg.params)
        probs = dict(zip(alg.bit_strs, probs))

    probs_plot = probs
    if sym:
        probs_plot = {k: v + probs[swap(k)] for k, v in probs.items() if k.startswith("0")}
    bar = bar_count(
        ax=axes[0],
        counts=probs_plot,
        sort_type="desc",
        bar_params={"color": "#333355"},
        x_rot=90,
    )
    red_inds = np.unique(
        np.append(
            np.where(bar.datavalues == probs_plot[correct_bs]),
            np.where(bar.datavalues == probs_plot.get(swap(correct_bs))),
        )
    ).numpy()
    for ind in red_inds:
        bar[ind].set_color("r")

    cost_min, cost_max = alg.costs.min(), alg.costs.max()

    axes[1].plot(alg.costs, lw=3, c="#333333")
    axes[1].set_ylabel("Cost", fontsize=13)
    axes[1].set_xlabel("Step", fontsize=13)
    axes[1].set_xlim(1, alg.evals)
    axes[1].yaxis.set_minor_locator(MultipleLocator(0.25))
    axes[1].yaxis.set_major_locator(MultipleLocator(0.5))
    axes[1].yaxis.set_major_formatter(StrMethodFormatter("{x:.1f}"))
    axes[1].axhline(cost_min, ls="dashed", c="k", alpha=0.4)
    axes[1].annotate(
        f"Minimum = {cost_min:.3f}",
        textcoords="axes fraction",
        xy=(0.9 * alg.evals, cost_min),
        xytext=(0.5, 0.2),
        fontsize=15,
        arrowprops=dict(facecolor="#666666", edgecolor="#333333", shrink=0.05, width=2),
        bbox=dict(fc="lightblue", ec="steelblue", lw=2, alpha=0.5),
    )
    axes[1].grid(alpha=0.4)
    axes[1].tick_params(top=True, right=True, which="both")
    axes[1].set_title("Cost function per step")

    fig.suptitle(
        f"{alg.__class__.__name__.replace('_', ' ')}",
        fontsize=22,
        x=0.55,
        y=0.85,
        bbox=dict(fc="#a9d3ff", ec="k", boxstyle="round4"),
    )
    axes[0].annotate(
        f"$p={alg.depth}$",
        xy=(0.538, 0.73),
        fontsize=15,
        xycoords="subfigure fraction",
        bbox=dict(fc="#a9d3ff", ec="k", boxstyle="round4", alpha=0.5),
    )


# FALQON Probabilities vs Depth

In [14]:
# Use event run from before
anim_probs_plot = []
for ind in range(falqon.depth):
    probs = falqon.depth_probs[ind]
    # Bitstrings are symmetric, so combine
    anim_probs_plot.append(numpy.array([v + probs[swap(k)] for k, v in probs.items() if k.startswith("0")]))

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
fig.set_facecolor("#d3d4d9")
rcParams["animation.embed_limit"] = 2**34


def animate(i):
    ax.cla()
    fig.suptitle(f"$\\Delta t={falqon.dt}$ | Depth: $p={i}$", fontsize=24)
    ax.set_facecolor("#d3d4d9")
    ax.set_ylim(0, np.ceil(100 * anim_probs_plot[-1].max()) / 100)
    ax.set_ylabel("Probability", fontsize=20)
    ax.tick_params(axis="x", rotation=45, labelsize=15)
    ax.tick_params(axis="y", labelsize=18)
    sorted_inds = np.argsort(anim_probs_plot[i])
    sorted_probs = anim_probs_plot[i][sorted_inds]
    sorted_bit_strs = falqon.bit_strs[:32][sorted_inds]
    colored_ind = np.where(sorted_bit_strs == "000111")[0][0]
    bar = ax.bar(sorted_bit_strs, sorted_probs, color="#4b88a2")
    bar[colored_ind].set_color("#bb0a21")


total = 30  # in seconds
anim = FuncAnimation(fig, animate, frames=falqon.depth, interval=(1000 * total) / falqon.depth)


HTML(anim.to_jshtml())


In [ ]:
# anim.save("test.mp4")
